In [1]:
%load_ext autoreload
%load_ext autotime

time: 319 µs (started: 2023-03-08 08:40:50 -08:00)


In [2]:
%run init.ipynb

/usr/workspace/iopp/.conda/envs/jupyter/lib/python3.9/site-packages/dask_jobqueue/core.py:17: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile, get_ip_interface
/usr/workspace/iopp/.conda/envs/jupyter/lib/python3.9/site-packages/dask_jobqueue/core.py:17: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile, get_ip_interface
/usr/workspace/iopp/.conda/envs/jupyter/lib/python3.9/site-packages/dask_jobqueue/htcondor.py:6: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import parse_bytes


{'cls': <class 'distributed.scheduler.Scheduler'>, 'options': {'protocol': 'tcp://', 'interface': None, 'host': None, 'dashboard_address': ':8787', 'security': None}}
time: 2.53 s (started: 2023-03-08 08:40:50 -08:00)


In [3]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

time: 16.9 ms (started: 2023-03-08 08:40:53 -08:00)


In [4]:
import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import json
import math
import numpy as np
import os
import pandas as pd
# import pyarrow as pa
from dask import compute, delayed
from dask.dataframe import DataFrame
from dask.distributed import Client, LocalCluster, fire_and_forget, wait, worker_client
from wisio.utils.file_utils import ensure_dir
from wisio.utils.json_encoders import NpEncoder
from wisio.utils.logger import create_logger, format_log

time: 617 ms (started: 2023-03-08 08:40:53 -08:00)


In [5]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.LSFCluster
Dashboard: http://192.168.130.70:8787/status,
Dashboard: http://192.168.130.70:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://192.168.130.70:35023,Workers: 0
Dashboard: http://192.168.130.70:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


time: 47.7 ms (started: 2023-03-08 08:40:53 -08:00)


In [6]:
import json

def load_global_min_max(log_dir: str):
    with open(f"{log_dir}/global.json") as file:
        global_min_max = json.load(file)
    return global_min_max

time: 982 µs (started: 2023-03-08 08:40:54 -08:00)


In [8]:
log_dir = "/p/gpfs1/iopp/recorder_app_logs/montage_pegasus/nodes-32/_parquet"
ddf = dd.read_parquet(f"{log_dir}/*.parquet")
ddf 

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
npartitions=36,,,,,,,,,,,,,,,,,,,,,
,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,object,int64,int32,float32,float32,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 212 ms (started: 2023-03-08 08:41:05 -08:00)


In [9]:
ddf.head()

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,...,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
0,1,132672,0,321552,0,3,2.137661,2.137760,fopen64,0,...,mProject,mProject#lassen192#132672#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,0,0,0.000000,0.000099,21377103,4007557435994691998,2303625805466626068
1,2,132672,0,321552,0,1,2.137835,2.138117,fread,0,...,mProject,mProject#lassen192#132672#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,2,0,0.006764,0.000282,21379759,4007557435994691998,2303625805466626068
2,3,132672,0,321552,0,3,2.138125,2.138141,fclose,0,...,mProject,mProject#lassen192#132672#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,0,0,0.000000,0.000016,21381327,4007557435994691998,2303625805466626068
3,4,132672,0,321552,0,3,2.138159,2.138171,fopen64,0,...,mProject,mProject#lassen192#132672#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,0,0,0.000000,0.000012,21381647,4007557435994691998,2303625805466626068
4,5,132672,0,321552,0,1,2.138231,2.138246,fread,0,...,mProject,mProject#lassen192#132672#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,2,0,0.127157,0.000015,21382382,4007557435994691998,2303625805466626068


time: 3.06 s (started: 2023-03-08 08:41:07 -08:00)


In [10]:
ddf.index.count().compute()

12346353

time: 2.33 s (started: 2023-03-08 08:41:15 -08:00)


In [11]:
ddf['tend'].max().compute()

433.1474304199219

time: 792 ms (started: 2023-03-08 08:41:18 -08:00)


In [12]:
ddf['file_id'].unique().count().compute()

19680

time: 636 ms (started: 2023-03-08 08:41:18 -08:00)


In [13]:
ddf['proc_id'].unique().count().compute()

11488

time: 576 ms (started: 2023-03-08 08:41:19 -08:00)


In [14]:
tmid_min, tmid_max = load_global_min_max(log_dir=log_dir)['tmid']
tmid_min, tmid_max

(0, 4331474299)

time: 3.33 ms (started: 2023-03-08 08:41:20 -08:00)


In [15]:
tmid_max / 1e7 / 60

7.219123831666667

time: 2.99 ms (started: 2023-03-08 08:41:26 -08:00)


In [16]:
trange = np.arange(tmid_min, tmid_max, 1e7)
ddf = ddf.map_partitions(lambda df: df.assign(trange=np.digitize(df['tmid'], trange, right=True)))

time: 12.3 ms (started: 2023-03-08 08:41:30 -08:00)


In [17]:
VIEW_TYPES = ['trange', 'file_id', 'proc_id']

time: 537 µs (started: 2023-03-08 08:41:31 -08:00)


In [18]:
DELTA_BINS = [
    0,
    0.01,
    0.1,
    0.25,
    0.5,
    0.75,
    1
]

def filter_delta(ddf: dd.DataFrame, delta: float, max_io_time: float, metric='duration'):
    def set_delta(df: pd.DataFrame):
        df['duration_csp'] = df['duration_sum'].cumsum() / max_io_time
        df['duration_delta'] = df['duration_csp'].diff().fillna(df['duration_csp'])
        df['duration_score'] = np.digitize(df['duration_delta'], bins=DELTA_BINS, right=True)
        df['duration_cut'] = np.choose(df['duration_score'] - 1, choices=DELTA_BINS, mode='clip')
        return df
    ddf = ddf.map_partitions(set_delta)
    return ddf[ddf['duration_delta'] > delta]

time: 1.96 ms (started: 2023-03-08 08:41:31 -08:00)


In [19]:
from copy import copy

view_types = copy(VIEW_TYPES)
view_types.append('io_cat')
view_types.append('acc_pat')

main_view = ddf[ddf['cat'] == 0] \
    .groupby(view_types) \
    .agg({
        'duration': [sum],
        'index': ['count'],
        'size': [min, max, sum],
        'file_name': ['first'],
        'proc_name': ['first']
#         'func_id': [unique_flatten()]
    }) \
    .persist()

main_view


Dask DataFrame Structure:
              duration  index   size               file_name proc_name
                   sum  count    min    max    sum     first     first
npartitions=1                                                         
               float32  int64  int64  int64  int64    object    object
                   ...    ...    ...    ...    ...       ...       ...
Dask Name: aggregate-agg, 1 tasks

time: 81.4 ms (started: 2023-03-08 08:41:34 -08:00)


In [20]:
wait(main_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('aggregate-agg-b685a01d03c46b8c18bb9a1e2c11d446', 0)>}, not_done=set())

time: 1.15 s (started: 2023-03-08 08:41:34 -08:00)


In [21]:
main_view = main_view.reset_index()
main_view.columns = ['_'.join(tup).rstrip('_') for tup in main_view.columns.values]
main_view = main_view.persist()

time: 28 ms (started: 2023-03-08 08:41:36 -08:00)


In [22]:
wait(main_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('rename-2ae52ef815e81246ed1f2972050fee1a', 0)>}, not_done=set())

time: 3.58 ms (started: 2023-03-08 08:41:37 -08:00)


In [23]:
main_view['read_count'] = 0
main_view['write_count'] = 0
main_view['metadata_count'] = 0
main_view['read_count'] = main_view['read_count'].mask(main_view['io_cat'] == 1, main_view['index_count'])
main_view['write_count'] = main_view['write_count'].mask(main_view['io_cat'] == 2,  main_view['index_count'])
main_view['metadata_count'] = main_view['metadata_count'].mask(main_view['io_cat'] == 3,  main_view['index_count'])

main_view['read_size'] = 0
main_view['write_size'] = 0
main_view['metadata_size'] = 0
main_view['read_size'] = main_view['read_size'].mask(main_view['io_cat'] == 1, main_view['size_sum'])
main_view['write_size'] = main_view['write_size'].mask(main_view['io_cat'] == 2,  main_view['size_sum'])
main_view['metadata_size'] = main_view['metadata_size'].mask(main_view['io_cat'] == 3,  main_view['size_sum'])

main_view['read_time'] = 0
main_view['write_time'] = 0
main_view['metadata_time'] = 0
main_view['read_time'] = main_view['read_time'].mask(main_view['io_cat'] == 1, main_view['duration_sum'])
main_view['write_time'] = main_view['write_time'].mask(main_view['io_cat'] == 2,  main_view['duration_sum'])
main_view['metadata_time'] = main_view['metadata_time'].mask(main_view['io_cat'] == 3,  main_view['duration_sum'])


main_view['sequential_count'] = 0
main_view['random_count'] = 0
main_view['sequential_count'] = main_view['sequential_count'].mask(main_view['acc_pat'] == 0, main_view['index_count'])
main_view['random_count'] = main_view['random_count'].mask(main_view['acc_pat'] == 1, main_view['index_count'])

main_view['sequential_size'] = 0
main_view['random_size'] = 0
main_view['sequential_size'] = main_view['sequential_size'].mask(main_view['acc_pat'] == 0, main_view['size_sum'])
main_view['random_size'] = main_view['random_size'].mask(main_view['acc_pat'] == 1, main_view['size_sum'])

main_view['sequential_time'] = 0
main_view['random_time'] = 0
main_view['sequential_time'] = main_view['sequential_time'].mask(main_view['acc_pat'] == 0, main_view['duration_sum'])
main_view['random_time'] = main_view['random_time'].mask(main_view['acc_pat'] == 1, main_view['duration_sum'])


main_view['data_count'] = main_view['write_count'] + main_view['read_count']
#main_view['metadata_count'] = main_view['metadata_count']
main_view['data_size'] = main_view['write_size'] + main_view['read_size']
main_view['data_time'] = main_view['write_time'] + main_view['read_time']

# main_view['workload_type'] = 0
# main_view['workload_type'] = main_view['workload_type'].mask(main_view['read_count'] == 0, 3)
# main_view['workload_type'] = main_view['workload_type'].mask(main_view['write_count'] == 0, 1)


final_view = main_view \
    .groupby(['trange', 'file_id', 'proc_id']) \
    .agg({
        'duration_sum': sum,
        'index_count': sum,
        'size_sum': sum,
        'read_count': sum,
        'write_count': sum,
        'metadata_count': sum,
        'read_size': sum,
        'write_size': sum,
        'metadata_size': sum,
        'read_time': sum,
        'write_time': sum,
        'metadata_time': sum,
        'sequential_count': sum,
        'random_count': sum,
        'sequential_size': sum,
        'random_size': sum,
        'sequential_time': sum,
        'random_time': sum,
        'data_count': sum,
        'data_size': sum,
        'data_time': sum,
#         'workload_type': sum,
        'size_min': min,
        'size_max': max,
        'file_name_first': 'first',
        'proc_name_first': 'first'
    }) \
    .persist()


time: 332 ms (started: 2023-03-08 08:41:38 -08:00)


In [24]:
wait(final_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('aggregate-agg-5861e161f65f5b94c4f75d51f5298a2a', 0)>}, not_done=set())

time: 3.86 ms (started: 2023-03-08 08:41:39 -08:00)


In [25]:
final_view = final_view \
    .reset_index() \
    .rename(columns={
        'file_name_first': 'file_name', 
        'proc_name_first': 'proc_name'
    }) \
    .persist()

time: 27.7 ms (started: 2023-03-08 08:41:40 -08:00)


In [26]:
wait(final_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('rename-035bde368a6597a3827f0be645d5906b', 0)>}, not_done=set())

time: 3.42 ms (started: 2023-03-08 08:41:40 -08:00)


In [27]:
final_view.head()

,trange,file_id,proc_id,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,...,random_size,sequential_time,random_time,data_count,data_size,data_time,size_min,size_max,file_name,proc_name
0,1,-9222633358449720911,-3892388038737598476,0.000678,1,0,0,0,1,0,...,0,0.000678,0.0,0,0,0.0,0,0,3-diff.000017.000048.fits,mDiffFit#lassen530#38672#35184372410352
1,1,-9221343239341138544,-3892500186846664716,0.000625,1,0,0,0,1,0,...,0,0.000625,0.0,0,0,0.0,0,0,2-fit.000005.000017.txt,mDiffFit#lassen522#135898#35184372410352
2,1,-9220642067073431512,-3892417125581330444,0.055775,1,0,0,0,1,0,...,0,0.055775,0.0,0,0,0.0,0,0,2-fit.000020.000022.txt,mDiffFit#lassen369#18451#35184372410352
3,1,-9219592665727134543,-3892417125581527052,0.003165,1,0,0,0,1,0,...,0,0.003165,0.0,0,0,0.0,0,0,2-fit.000020.000032.txt,mDiffFit#lassen369#18448#35184372410352
4,1,-9217476538956658760,984959368281188372,0.010928,3,0,0,0,3,0,...,0,0.010928,0.0,0,0,0.0,0,0,cposs2ukstu_blue_005_008.fits,mBackground#lassen676#176277#35184372410384


time: 38.4 ms (started: 2023-03-08 08:41:41 -08:00)


In [28]:
del main_view

time: 779 µs (started: 2023-03-08 08:41:42 -08:00)


In [29]:
final_view.head()

,trange,file_id,proc_id,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,...,random_size,sequential_time,random_time,data_count,data_size,data_time,size_min,size_max,file_name,proc_name
0,1,-9222633358449720911,-3892388038737598476,0.000678,1,0,0,0,1,0,...,0,0.000678,0.0,0,0,0.0,0,0,3-diff.000017.000048.fits,mDiffFit#lassen530#38672#35184372410352
1,1,-9221343239341138544,-3892500186846664716,0.000625,1,0,0,0,1,0,...,0,0.000625,0.0,0,0,0.0,0,0,2-fit.000005.000017.txt,mDiffFit#lassen522#135898#35184372410352
2,1,-9220642067073431512,-3892417125581330444,0.055775,1,0,0,0,1,0,...,0,0.055775,0.0,0,0,0.0,0,0,2-fit.000020.000022.txt,mDiffFit#lassen369#18451#35184372410352
3,1,-9219592665727134543,-3892417125581527052,0.003165,1,0,0,0,1,0,...,0,0.003165,0.0,0,0,0.0,0,0,2-fit.000020.000032.txt,mDiffFit#lassen369#18448#35184372410352
4,1,-9217476538956658760,984959368281188372,0.010928,3,0,0,0,3,0,...,0,0.010928,0.0,0,0,0.0,0,0,cposs2ukstu_blue_005_008.fits,mBackground#lassen676#176277#35184372410384


time: 36.2 ms (started: 2023-03-08 08:41:42 -08:00)


In [30]:
DELTA_BINS = [
    0,
    0.01,
    0.1,
    0.25,
    0.5,
    0.75,
    1
]

def filter_delta(view_type: str, ddf: dd.DataFrame, delta: float, metric: str, max_io_time: dd.core.Scalar):
    metric_col, csp_col, delta_col, score_col, cut_col = (
        f"{metric}_sum",
        f"{metric}_csp",
        f"{metric}_delta",
        f"{metric}_score",
        f"{metric}_cut"
    )

    def set_delta(df: pd.DataFrame):
        df[csp_col] = df[metric_col].cumsum() / max_io_time
        df[delta_col] = df[csp_col].diff().fillna(df[csp_col])
        df[score_col] = np.digitize(df[delta_col], bins=DELTA_BINS, right=True)
        df[cut_col] = np.choose(df[score_col] - 1, choices=DELTA_BINS, mode='clip')
        return df

    index = pd.MultiIndex(levels=[[1]], codes=[[]], names=[view_type])
    meta = dd.utils.make_meta(([
        (metric_col, np.float64),
        (csp_col, np.float64),
        (delta_col, np.float64),
        (score_col, np.int8),
        (cut_col, np.float16),
    ]), index=index)

#     ddf = ddf.map_partitions(set_delta, meta=meta)
    ddf = ddf.map_partitions(set_delta)

    return ddf[ddf[delta_col] > delta]


time: 1.87 ms (started: 2023-03-08 08:41:43 -08:00)


In [31]:
max_io_time = final_view.groupby(['proc_id']).agg({'duration_sum':sum})['duration_sum'].max()
max_io_time

dd.Scalar<series-..., dtype=float64>

time: 12.8 ms (started: 2023-03-08 08:41:44 -08:00)


In [37]:
DELTA = 0.0001
CUT = 0.5

time: 801 µs (started: 2023-03-08 08:44:41 -08:00)


In [71]:
import itertools as it

def _set_metric_percentages(ddf: dd.DataFrame, max_io_time: dd.core.Scalar, metric: str):
    metric_col, pero_col, perr_col = (
        f"{metric}_sum",
        f"{metric}_pero",
        f"{metric}_perr"
    )
    ddf[pero_col] = ddf[metric_col] / max_io_time
    ddf[perr_col] = ddf[metric_col] / ddf[metric_col].sum()
    return ddf

def _compute_expanded_view(
    parent_view: dd.DataFrame,
    view_type: str,
    cols: tuple,
    metric: str,
    delta: float,
    max_io_time: dd.core.Scalar
):
    # Get column names
    metric_col, score_col, cut_col = cols
    # Check view type
    if view_type is not 'proc_id':
        # Compute `proc_id` view first
        group_view = parent_view \
            .groupby([view_type, 'proc_id']) \
            .agg({metric_col: sum}) \
            .groupby([view_type]) \
            .max()
    else:
        # Compute group view
        group_view = parent_view \
            .groupby([view_type]) \
            .agg({metric_col: sum})
    # Filter view
    filtered_view = filter_delta(
        view_type=view_type,
        ddf=group_view,
        delta=delta,
        metric=metric,
        max_io_time=max_io_time
    )
    # Get score view
    score_view = filtered_view.reset_index()[[view_type, score_col, cut_col]]
    # Find filtered records and set duration scores
    expanded_view = parent_view \
        .query(f"{view_type} in @indices", local_dict={'indices': filtered_view.index.unique()}) \
        .drop(columns=[score_col, cut_col], errors='ignore') \
        .merge(score_view, on=[view_type])    
    # Set metric percentages
    expanded_view = _set_metric_percentages(ddf=expanded_view, max_io_time=max_io_time, metric=metric)
    # Return expanded view
    return expanded_view

def _view_permutations(r: int):
    return it.permutations(VIEW_TYPES, r + 1)

metric = 'duration'
views = {}
for view_permutation in it.chain.from_iterable(map(_view_permutations, range(len(VIEW_TYPES)))):
    # Read types
    parent_type = view_permutation[:-1]
    view_type = view_permutation[-1]
    # Get parent view
    parent_view = views[parent_type]['expanded_view'] if parent_type in views else final_view
    # Create colum names
    metric_col, score_col, cut_col = f"{metric}_sum", f"{metric}_score", f"{metric}_cut"
    # Compute expanded view
    expanded_view = _compute_expanded_view(
        parent_view=parent_view,
        view_type=view_type,
        cols=(metric_col, score_col, cut_col),
        metric=metric,
        delta=DELTA,
        max_io_time=max_io_time
    )
    # Compute bottleneck view
    bottleneck_view = expanded_view[expanded_view[cut_col] >= CUT]
    # Set duration scores to expanded view
#     bottleneck_view = filter_delta(
#         view_type=view_type,
#         ddf=bottleneck_view,
#         delta=0,
#         metric=metric,
#         max_io_time=max_io_time
#     )
    # Return views
    views[view_permutation] = {}
    views[view_permutation]['expanded_view'] = expanded_view
    views[view_permutation]['bottleneck_view'] = bottleneck_view

len(views)

15

time: 2.1 s (started: 2023-03-08 09:10:03 -08:00)


In [72]:
for view_key, view_dict in views.items():
    print(view_key, 'expanded_view', len(view_dict['expanded_view']), 'bottleneck_view', len(view_dict['bottleneck_view']))

('trange',) expanded_view 62779 bottleneck_view 0
('file_id',) expanded_view 43702 bottleneck_view 0
('proc_id',) expanded_view 60743 bottleneck_view 1610
('trange', 'file_id') expanded_view 43703 bottleneck_view 0
('trange', 'proc_id') expanded_view 60699 bottleneck_view 1610
('file_id', 'trange') expanded_view 43701 bottleneck_view 0
('file_id', 'proc_id') expanded_view 41711 bottleneck_view 1604
('proc_id', 'trange') expanded_view 60733 bottleneck_view 0
('proc_id', 'file_id') expanded_view 41738 bottleneck_view 0
('trange', 'file_id', 'proc_id') expanded_view 41722 bottleneck_view 1604
('trange', 'proc_id', 'file_id') expanded_view 41706 bottleneck_view 0
('file_id', 'trange', 'proc_id') expanded_view 41725 bottleneck_view 1604
('file_id', 'proc_id', 'trange') expanded_view 41710 bottleneck_view 0
('proc_id', 'trange', 'file_id') expanded_view 41741 bottleneck_view 0
('proc_id', 'file_id', 'trange') expanded_view 41737 bottleneck_view 0
time: 10.6 s (started: 2023-03-08 09:10:05 -0

In [73]:
import dask.dataframe as dd
import pandas as pd
from dask import compute, delayed
from dask.distributed import get_client
from typing import Any, Dict

BOTTLENECK_ORDER = dict(
    file_id=('file_name', 'proc_name', 'trange'),
    proc_id=('proc_name', 'trange', 'file_name'),
    trange=('trange', 'proc_name', 'file_name'),
)
BOTTLENECK_TYPE = dict(
    file_id='file_name',
    proc_id='proc_name',
    trange='trange'
)

def _calculate_llc(level_row: pd.Series):
    llc = dict(level_row)
    for view_type in VIEW_TYPES:
        if view_type in llc:
            llc.pop(view_type)
        bottleneck_type = BOTTLENECK_TYPE[view_type]
        if bottleneck_type in llc:
            llc.pop(bottleneck_type)
    return llc


@delayed
def _process_bottleneck_view(
    view_key: tuple,
    ll_view: pd.DataFrame,
    ml_view: pd.DataFrame,
    hl_view: pd.DataFrame,
):
    # Get view type
    view_type = view_key[-1]
    # Get ordered bottleneck columns
    _, ml_col, ll_col = BOTTLENECK_ORDER[view_type]
    # Init bottlenecks
    bottlenecks = {}
    # Loop through index tuples
    ids_tuple = ll_view.index
    for hl_id, ml_id, ll_id in ids_tuple:
        hl_row = hl_view.loc[hl_id]
        ml_row = ml_view.loc[(hl_id, ml_id)]
        ll_row = ll_view.loc[(hl_id, ml_id, ll_id)]
        if hl_id not in bottlenecks:
            bottlenecks[hl_id] = {}
            bottlenecks[hl_id]['llc'] = _calculate_llc(hl_row)
            bottlenecks[hl_id][ml_col] = {}
        if ml_id not in bottlenecks[hl_id][ml_col]:
            bottlenecks[hl_id][ml_col][ml_id] = {}
            bottlenecks[hl_id][ml_col][ml_id]['llc'] = _calculate_llc(ml_row)
            bottlenecks[hl_id][ml_col][ml_id][ll_col] = {}
        if ll_id not in bottlenecks[hl_id][ml_col][ml_id][ll_col]:
            bottlenecks[hl_id][ml_col][ml_id][ll_col][ll_id] = {}
            bottlenecks[hl_id][ml_col][ml_id][ll_col][ll_id]['llc'] = _calculate_llc(ll_row)
    # Return view key & bottlenecks
    return view_key, bottlenecks

def detect_bottlenecks(views: Dict[tuple, dd.DataFrame], view_types: list) -> Dict[tuple, object]:
    # Keep bottleneck views
    bottleneck_views = {}
    # Run through views
    for view_key, view_dict in views.items():
        # Generate bottleneck views
        bottleneck_views[view_key] = _generate_bottlenecks_views(
            view_key=view_key,
            view_dict=view_dict,
            view_types=view_types,
        )
    # Generate bottlenecks
#     bottlenecks = _process_bottleneck_views(bottleneck_views=bottleneck_views)
    # Return bottleneck views
    return bottleneck_views

def _process_bottleneck_views(bottleneck_views: Dict[tuple, pd.DataFrame]):
    # Init bottlenecks
    bottlenecks = {}
    bottlenecks_delayed = []
    # Run through bottleneck views
    for view_key, view_dict in bottleneck_views.items():
        bottlenecks_delayed.append(_process_bottleneck_view(
            view_key=view_key,
            ll_view=view_dict['low_level_view'],
            ml_view=view_dict['mid_level_view'],
            hl_view=view_dict['high_level_view']
        ))
    # Compute all bottlenecks
    futures = compute(*bottlenecks_delayed, sync=False)
    results = get_client().gather(list(futures))
    for view_key, result in results:
        bottlenecks[view_key] = result
    # Return all bottlenecks
    return bottlenecks

def _generate_bottlenecks_views(
        view_key: tuple,
        view_dict: Dict[str, dd.DataFrame],
        view_types: list
    ):
        # Get view type
        view_type = view_key[-1]
        bottleneck_type = BOTTLENECK_TYPE[view_type]

        # Get parent view
        bottleneck_view = view_dict['bottleneck_view']

        # Create lower level view
        low_level_view = bottleneck_view \
            .groupby(list(BOTTLENECK_ORDER[view_type])) \
            .first() \
            .drop(columns=['acc_pat', 'io_cat'], errors='ignore')

        # Non-proc agg columns
        non_proc_agg_dict = _get_agg_dict(view_types=view_types, view_columns=low_level_view.columns, is_proc=False)
        proc_agg_dict = _get_agg_dict(view_types=view_types, view_columns=low_level_view.columns, is_proc=True)

        if bottleneck_type is not 'proc_name':

            mid_level_view = low_level_view \
                .reset_index() \
                .groupby([bottleneck_type, 'proc_name']) \
                .agg(non_proc_agg_dict)

            high_level_view = mid_level_view \
                .reset_index() \
                .groupby([bottleneck_type]) \
                .agg(proc_agg_dict)

        else:

            mid_level_view = low_level_view \
                .reset_index() \
                .groupby([bottleneck_type, 'trange']) \
                .agg(non_proc_agg_dict)

            high_level_view = mid_level_view \
                .reset_index() \
                .groupby([bottleneck_type]) \
                .agg(non_proc_agg_dict)

        return dict(
            low_level_view=low_level_view.persist(),
            mid_level_view=mid_level_view.persist(),
            high_level_view=high_level_view.persist()
        )

def _get_agg_dict(view_types: list, view_columns: list, is_proc=False):
    if is_proc:
        agg_dict = {col: max if any(x in col for x in 'duration time'.split()) else sum for col in view_columns}
    else:
        agg_dict = {col: sum for col in view_columns}
#     agg_dict['func_id'] = unique_flatten()
    agg_dict['size_min'] = min
    agg_dict['size_max'] = max
    agg_dict['duration_cut'] = 'first'
    agg_dict['duration_score'] = 'first'
    for view_type in view_types:
        if view_type in agg_dict:
            agg_dict.pop(view_type)
        bottleneck_type = BOTTLENECK_TYPE[view_type]
        if bottleneck_type in agg_dict:
            agg_dict.pop(bottleneck_type)
    return agg_dict


time: 8.51 ms (started: 2023-03-08 09:10:16 -08:00)


In [74]:
bottleneck_views = detect_bottlenecks(
    views=views,
    view_types=VIEW_TYPES,
)

len(bottleneck_views)

15

time: 3.53 s (started: 2023-03-08 09:10:16 -08:00)


In [75]:
for view_key, bottleneck_view_dict in bottleneck_views.items():
    print(view_key, 'hl_view', len(bottleneck_view_dict['high_level_view']), 'ml_view', len(bottleneck_view_dict['mid_level_view']), 'll_view', len(bottleneck_view_dict['low_level_view']))

('trange',) hl_view 0 ml_view 0 ll_view 0
('file_id',) hl_view 0 ml_view 0 ll_view 0
('proc_id',) hl_view 3 ml_view 55 ll_view 1610
('trange', 'file_id') hl_view 0 ml_view 0 ll_view 0
('trange', 'proc_id') hl_view 3 ml_view 55 ll_view 1610
('file_id', 'trange') hl_view 0 ml_view 0 ll_view 0
('file_id', 'proc_id') hl_view 3 ml_view 55 ll_view 1604
('proc_id', 'trange') hl_view 0 ml_view 0 ll_view 0
('proc_id', 'file_id') hl_view 0 ml_view 0 ll_view 0
('trange', 'file_id', 'proc_id') hl_view 3 ml_view 55 ll_view 1604
('trange', 'proc_id', 'file_id') hl_view 0 ml_view 0 ll_view 0
('file_id', 'trange', 'proc_id') hl_view 3 ml_view 55 ll_view 1604
('file_id', 'proc_id', 'trange') hl_view 0 ml_view 0 ll_view 0
('proc_id', 'trange', 'file_id') hl_view 0 ml_view 0 ll_view 0
('proc_id', 'file_id', 'trange') hl_view 0 ml_view 0 ll_view 0
time: 4.19 s (started: 2023-03-08 09:10:19 -08:00)


In [76]:
bottleneck_views[('trange',)]['high_level_view'].compute()

,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,write_size,metadata_size,read_time,...,random_time,data_count,data_size,data_time,size_min,size_max,duration_score,duration_cut,duration_pero,duration_perr
trange,,,,,,,,,,,,,,,,,,,,,


time: 23.7 ms (started: 2023-03-08 09:10:24 -08:00)


In [77]:
bottleneck_views[('file_id',)]['high_level_view'].compute()

,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,write_size,metadata_size,read_time,...,random_time,data_count,data_size,data_time,size_min,size_max,duration_score,duration_cut,duration_pero,duration_perr
file_name,,,,,,,,,,,,,,,,,,,,,


time: 18.7 ms (started: 2023-03-08 09:10:24 -08:00)


In [78]:
bottleneck_views[('proc_id',)]['high_level_view'].compute()

,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,write_size,metadata_size,read_time,...,random_time,data_count,data_size,data_time,size_min,size_max,duration_score,duration_cut,duration_pero,duration_perr
proc_name,,,,,,,,,,,,,,,,,,,,,
mAdd#lassen677#17750#35184372410384,3.628076,250846,3999379634,164116,86402,328,3169932994,829445760,880,3.015923,...,0.000010,250518,3999378754,3.618456,0,23040,6,0.75,0.970282,0.015538
mAdd#lassen731#174503#35184372410384,3.650154,251564,4028580224,164834,86402,328,3199133586,829445760,878,3.023329,...,0.000009,251236,4028579346,3.640688,0,23040,6,0.75,0.976187,0.015632
mAdd#lassen734#63830#35184372410384,3.739198,251798,4037606516,165068,86402,328,3208159874,829445760,882,3.109872,...,0.000008,251470,4037605634,3.729229,0,23040,6,0.75,1.000000,0.016014


time: 28.6 ms (started: 2023-03-08 09:10:24 -08:00)


In [79]:
bottleneck_views[('proc_id',)]['mid_level_view'].compute()

duration_sum  index_count  \
proc_name                            trange                              
mAdd#lassen677#17750#35184372410384  1           0.106461           89   
                                     2           0.179831        12121   
                                     3           0.154683        12217   
                                     4           0.128478         7563   
                                     5           0.236454        18809   
                                     6           0.282696        19034   
                                     7           0.200355        17246   
                                     8           0.171108         7790   
                                     9           0.240366        19020   
                                     10          0.286649        19149   
                                     11          0.195538        16749   
                                     12          0.151182         5623   
                                     13          0.235700        19036   
                                     14          0.272563        19194   
                                     15          0.218296        18789   
                                     16          0.158770         9442   
                                     17          0.237724        18989   
                                     18          0.171222         9986   
mAdd#lassen731#174503#35184372410384 1           0.092928           89   
                                     2           0.188950        13159   
                                     3           0.150909        11063   
                                     4           0.182895        12336   
                                     5           0.241130        18651   
                                     6           0.276142        19755   
                                     7           0.179291        12136   
                                     8           0.169322        11779   
                                     9           0.242271        18832   
                                     10          0.275013        19822   
                                     11          0.162648        12367   
                                     12          0.154740         9537   
                                     13          0.241936        18852   
                                     14          0.293782        18934   
                                     15          0.189401        15499   
                                     16          0.117440         5735   
                                     17          0.238116        18920   
                                     18          0.253240        14098   
mAdd#lassen734#63830#35184372410384  1           0.079273           89   
                                     2           0.214108        14301   
                                     3           0.133690         9823   
                                     4           0.140877         8117   
                                     5           0.224158        17402   
                                     6           0.276167        17406   
                                     7           0.234536        19516   
                                     8           0.147009         6725   
                                     9           0.226358        17479   
                                     10          0.279494        17140   
                                     11          0.230784        19506   
                                     12          0.140883         6861   
                                     13          0.226295        17475   
                                     14          0.278066        16912   
                                     15          0.230867        19444   
                                     16          0.139066         7051   
                                     17          

time: 68.3 ms (started: 2023-03-08 09:10:24 -08:00)


In [80]:
proc_id_bview = bottleneck_views[('proc_id',)]['low_level_view'].compute()
proc_id_bview = proc_id_bview.drop(columns=['file_id', 'proc_id'])
proc_id_bview

duration_sum  \
proc_name                           trange file_name                                                          
mAdd#lassen677#17750#35184372410384 1      /p/gpfs1/haridev/iopp/montage-workflow/scratch/...      0.000075   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...      0.005862   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...      0.023376   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...      0.005659   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...      0.012227   
...                                                                                                     ...   
mAdd#lassen734#63830#35184372410384 19     /p/gpfs1/haridev/iopp/montage-workflow/scratch/...      0.001226   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...      0.001291   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...      0.001212   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...      0.000922   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...      0.001106   

                                                                                               index_count  \
proc_name                           trange file_name                                                         
mAdd#lassen677#17750#35184372410384 1      /p/gpfs1/haridev/iopp/montage-workflow/scratch/...            4   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...            6   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...            6   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...            6   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...            6   
...                                                                                                    ...   
mAdd#lassen734#63830#35184372410384 19     /p/gpfs1/haridev/iopp/montage-workflow/scratch/...           85   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...           85   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...           85   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...           85   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...           85   

                                                                                               size_sum  \
proc_name                           trange file_name                                                      
mAdd#lassen677#17750#35184372410384 1      /p/gpfs1/haridev/iopp/montage-workflow/scratch/...       880   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...     23402   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...     23402   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...     23386   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...     23386   
...                                                                                                 ...   
mAdd#lassen734#63830#35184372410384 19     /p/gpfs1/haridev/iopp/montage-workflow/scratch/...   1753720   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...   1753720   
                                           /p/gpfs1/haridev/iopp/montage-workflow/scratch/...   1753720   
                                           

time: 48 ms (started: 2023-03-08 09:10:24 -08:00)
